In [ ]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from rag_service import RAGService
from config import (
    LLM_PROVIDER, OLLAMA_MODEL, OLLAMA_BASE_URL, 
    OPENAI_API_KEY, OPENAI_MODEL, LLM_TEMPERATURE
)
import os

In [2]:
# Initialization (LLM and RAG Service)
print(f"Current LLM Provider: {LLM_PROVIDER}")

# Initialize LLM based on configuration
if LLM_PROVIDER == "openai":
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY is required for OpenAI provider.")
    print(f"Initializing OpenAI LLM with model: {OPENAI_MODEL}")
    llm = ChatOpenAI(
        model=OPENAI_MODEL,
        temperature=LLM_TEMPERATURE,
        api_key=OPENAI_API_KEY
    )
else:
    # Default to Ollama
    print(f"Initializing Ollama LLM with model: {OLLAMA_MODEL}")
    llm = ChatOllama(
        model=OLLAMA_MODEL,
        temperature=LLM_TEMPERATURE, 
        base_url=OLLAMA_BASE_URL
    )

# Initialize RAG Service
rag_service = RAGService()
print("RAG Service Initialized.")

Current LLM Provider: ollama
Initializing Ollama LLM with model: qwen2.5:7b-instruct
Loading Embedding Model: text-embedding-3-large...
Loading FAISS Vector Store from vector_store...
RAG Service Initialized Successfully.
RAG Service Initialized.


In [6]:
# Direct Chat Execution
query = "سقف تعداد واحد در ترم کارآموزی چقدره؟"
print(f"Query: {query}")

messages = [HumanMessage(content=query)]
response = llm.invoke(messages)

print("\n--- Direct Chat Response ---")
print(response.content)

Query: سقف تعداد واحد در ترم کارآموزی چقدره؟

--- Direct Chat Response ---
در معمولاً، تعداد واحد در ترم کارآموزی برای هر دانشجو متفاوت است و این تعداد معمولاً توسط دانشگاه یا مؤسسه آموزشی تعیین می‌شود. برخی از دانشگاه‌ها ممکن است 12 تا 16 واحد را در یک ترم کارآموزی تعیین کنند، در حالی که سایر دانشگاه‌ها ممکن است این تعداد را کمتر یا بیشتر قرار دهند. برای جزئیات دقیق و مناسب، بهتر است با برنامه‌ریزی آموزشی یا مرکز مشاوره دانشگاه خود تماس بگیرید تا بدانید که چقدر واحد در ترم کارآموزی شما قرار دارد.


In [7]:
# RAG Retrieval Execution Only
query = "شرایط مشروطی در کارشناسی چیست؟"
print(f"Query: {query}\n")

# Use the internal retrieve method to inspect docs
docs = rag_service._retrieve_documents(query)

print(f"--- Retrieved {len(docs)} Documents ---\n")
for i, doc in enumerate(docs):
    source = doc.metadata.get("title", "Unknown Source")
    content = doc.page_content.replace("\n", " ").strip()[:200] + "..." # Truncate for display
    print(f"[Source {i+1}: {source}]")
    print(f"Content Preview: {content}\n")

Query: شرایط مشروطی در کارشناسی چیست؟

--- Retrieved 5 Documents ---

[Source 1: آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف]
Content Preview: 16. درس جبرانی: درسی است که با تأیید گروه آموزشی، گذراندن آن برای رفع کمبود دانش یا مهارت دانشجو، در آغاز دوره تحصیلی، ضروری تشخیص داده می شود. این نوع درس فقط برای دانشجویانی که از رشته های غیرمرتبط ...

[Source 2: آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف (ورودی‌های ۱۴۰۱ و ماقبل)]
Content Preview: ماده ۱۵- چنانچه میانگین نمرات دانشجو در هر نیم سال تحصیلی کمتر از ۱۲ باشد، دانشجو در آن نیم سال مشروط تلقی می شود. سقف مجاز مشروط اعم از متوالی یا متناوب در دوره کاردانی و کارشناسی ناپیوسته دو نیم سال...

[Source 3: آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف (ورودی‌های ۱۴۰۱ و ماقبل)]
Content Preview: 15. مشروطی:  وضعیت تحصیلی است که دانشجو در یک نیم سال معدل کمتر از حد نصاب نمره تعیین شده را کسب کرده باشد. برای دوره های کاردانی و کارشناسی این حد نصاب نمره ۱۲ (از ۲۰) است. 16. رشته تحصیلی:  یکی از ش...

[Source 4: آیین‌نامه آموزشی 

In [8]:
# Full RAG Pipeline Execution (Retrieve + Generate)
query = "آیین‌نامه غیبت در امتحانات چیست؟"
print(f"Query: {query}\n")

# 1. Retrieve & Augment
system_instruction, docs = rag_service.generate_augmented_prompt(query)

if not system_instruction:
    print("No relevant documents found.")
else:
    print(system_instruction)
    print(f"\n\n--- Retrieved {len(docs)} Documents ---\n")
    for i, doc in enumerate(docs):
        source = doc.metadata.get("title", "Unknown Source")
        content = doc.page_content.replace("\n", " ").strip()[:200] + "..." # Truncate for display
        print(f"[Source {i+1}: {source}]")
        print(f"Content Preview: {content}\n")

    # 2. Construct Messages
    messages = [
        SystemMessage(content=system_instruction),
        HumanMessage(content=query)
    ]

    # 3. Generate Response
    response = llm.invoke(messages)
    
    print("--- LLM Response ---")
    print(response.content)
    
    print("\n--- Sources ---")
    sources = list(set([doc.metadata.get("title", "Unknown") for doc in docs]))
    for s in sources:
        print(f"- {s}")

Query: آیین‌نامه غیبت در امتحانات چیست؟

Found 5 relevant documents.

--- RAG Response ---
آیین‌نامه غیبت در امتحانات شامل قوانین و مقررات مربوط به حضور دانشجویان در جلسات امتحانات و تأثیر غیبت بر نمرات و کارنامه دانشجویان است. برخی نقاط مهم آیین‌نامه شامل:

1. غیبت غیرموجه در امتحان منجر به نمره صفر در آن امتحان می‌شود.
2. مسافرت طولانی در طول امتحانات پایان نیم سال، دلیل موجهی برای غیبت نخواهد بود و غیبت غیرموجه خواهد بود.
3. به دلیل بیماری قادر به شرکت در امتحان نباشند، باید فقط به بهداری دانشگاه مراجعه کرده و توسط پزشک دانشگاه معاینه و در صورت لزوم گواهی لازم صادر گردد.
4. غیبت موجه در امتحان پایان نیم سال، با تشخیص شورای آموزشی دانشگاه، آن درس از مجموع درس‌ها حذف می‌شود و نمره نهایی براساس سوابق تحصیلی دانشجو محاسبه خواهد شد.
5. غیبت موجه به دلایل پزشکی، آن درس بشکل حذف W در کارنامه ثبت می‌شود و امکان اخذ امتحان مجدد وجود ندارد.

این قوانین در متون موجود توضیح داده شده‌اند.

--- Sources ---
- آیین‌نامه برگزاری و غیبت در امتحانات
- آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف 